# Counterfactual explanations
Counterfactual explanations (CEs) are an important tool from the field of explainable artificial intelligence (XAI). 
This notebook teaches what CEs are, why they are important, and provides how they can be discovered.

## To begin with: What is *XAI*?
XAI is a subfield of AI concerned with developing methods to help us use AI systems in a fair, safe, and responsible manner.
To do that, XAI aims at *explaining* why an AI system (typically, actually a machine learning model) behaves the way it does.
There are two main categories of XAI methods:

1 - Methods to understand why very large and complex models, like deep neural nets and large ensembles of decision trees, come to certain decisions/predictions. 
These models are typically called *black-box* models.

2 - Methods to generate models that are so simple that they can be interpreted directly. Models of this type are, e.g., decision trees, rule sets, and equations found by symbolic regression.
These models are typically called *glass-box* models.

## A brief intro to CEs
CEs belong to the first category mentioned above: methods to explain black-box models.
Let us consider the case in which we have a model that is a classifier, i.e., our model is a function $$f : \Omega^d → \mathbb{C},$$
where $\Omega^d$ is our space of $d$ features (some of which are numerical and thus in $\mathbb{R}$, some of which are categorical) while $\mathbb{C}$ is the space of classes (for example for a classifier of credit risk, $\mathbb{C} = \{ \textit{High risk}, \textit{Low risk} \}$).

Say $\mathbf{x} \in \Omega^d$ is a possible input for our classifier $f$.
$\mathbf{x}$ represents a user. For example, $\mathbf{x}$ can be the:
$$\mathbf{x} = ( \textit{ age : 22, gender : Female, savings : 5.000\$, job : student, } \dots ). $$
For a given $\mathbf{x}$, $f$ will predict a certain class $c$ (e.g., "$\textit{High risk}$").
Now, a CE aims to answer the question:
"What **small change** is needed to $\mathbf{x}$ such that the new input $\mathbf{x}^\prime$ will cause $f$ to produce the desired class $c^\star$? (e.g., $f(\mathbf{x}^\prime) = \textit{Low risk}$).

A CE is a possible answer to the question above. 
For example, an answer could be that the user needs to increase their savings ($\textit{5.000\$} → {8.000\$}$) and change occupation ($ \textit{student} \rightarrow \textit{part-time employed}$).
However, a CE may also reveal that $f$ changes its prediction based on ethnicity or gender (all other features remaining the same), meaning that $f$ learned harmful biases (e.g., from historical data) that perpetuate a discrimination against minorities (unfairness).

Here's a simplified depiction in a 2D feature space:
![](https://drive.google.com/uc?export=view&id=1eQTEExQhIgi-2sEoCcyMELfKXACTrxAW)


### Seeking *small* changes to $x$

We seek *small* changes to $x$ to observe how $f$ behaves in the neighborhood of an input to gain information on what the decision boundary looks like in that area. 
Moreover, a very interesting property of CEs is that they prescribe a possible intervention that the user may actually want to pursue!
Thus, we wish that the cost of intervention is small for the user. 
This means that $\mathbf{x}^\prime$ needs to be as close as possible to $\mathbf{x}$, under some meaningful distance function $\delta$ that captures the cost of intervention.

## Additional reading material
An excellent and beginner-friendly starting point is the book by Christoph Molnar: "Inteprable ML Book".
Here's a direct link to his chapter on CEs (co-written by Susanne Dandl): https://christophm.github.io/interpretable-ml-book/counterfactual.html

## Note: CEs vs adversarial examples
CEs are similar to adversarial examples (AEs). In both cases, one searches for changes to the input $x$ that trigger a change to the prediction made by $f$. However, CEs are intended to explain $f$ and not to fool it!

## Let's get started
In this notebook we simulate a financial credit risk situation, in which a black-box model (we will be using a random forest) has been trained to tell which users are at high or low risk of default (i.e., become unable of paying back the credit given by the bank).
We will then use a CE discovery algorithm to see how an user can change their (unfavorable) situation (i.e., f(x)=high risk).

### Set up libraries & random seed

In [ ]:
import numpy as np#用来处理数据或矩阵的库
import pandas as pd#解决数据分析任务，提供了大量能使我们快速便捷地处理数据的函数和方法
from sklearn.ensemble import RandomForestClassifier#随机森林算法
from sklearn.model_selection import train_test_split#将数据划分训练集和测试集
from sklearn.metrics import accuracy_score, balanced_accuracy_score#准确率

SEED = 42
np.random.seed(SEED) #为了再现性好

### Load data
We load the data set "South German Credit", which concerns learning a model of whether providing a financial credit to a user may be safe or risky.
See https://archive.ics.uci.edu/ml/datasets/South+German+Credit+%28UPDATE%29 for more info.

We get this data from the repo of CoGS, a baseline algorithm for the discovery of CEs (more details later). 

In [ ]:
#克隆repo
! git clone https://github.com/marcovirgolin/cogs
% cd /content/cogs#访问
! pip install .#安装repo

Cloning into 'cogs'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 37 (delta 12), reused 20 (delta 3), pack-reused 0
Unpacking objects: 100% (37/37), done.
/content/cogs
Processing /content/cogs
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for cogs: filename=cogs-1.0.0-py3-none-any.whl size=23323 sha256=bfbcbe6a8d379e20ad7023d11586ef90bafcb3d1558be0c07e1b581cfe64165a
  Stored in directory: /tmp/pip-ephem-wheel-cache-uojmy6do/wheels/7a/25/e6/fef7467ff3dd1da42831774b151adc77e68a0d3ca439f9a2a4
Successfully built cogs


Load the data and preprocess it a bit

In [ ]:
#加载数据集并进行一些预处理
df = pd.read_csv("south_german_credit.csv")#读取south_german_credit.csv文件的数据，并转换成DataFrame对象
df.drop("account_check_status",axis=1,inplace=True)#在df中删除account_check_status这一列，原数组名对应的内存值改变
categorical_feature_names = ['purpose', 'personal_status_sex',
    'other_debtors', 'other_installment_plans', 'telephone', 'foreign_worker']
#注意：其他一些特征是索引，为了简单起见，此处将其视为数字
label_name = 'credit_risk'#标签名设为credit_risk
desired_class = 1 #desired_class为1表示为低风险

for feat in categorical_feature_names: #将categorical features转换为整数型代码
    df[feat] = pd.Categorical(df[feat])#提取df[feat]列表数据的所有不重复的值
    df[feat] = df[feat].cat.codes#将df[feat]的字符类型用整数编码代替 
feature_names = list(df.columns)#转成列表形式
feature_names.remove(label_name)#对feature_names列表的label_name进行删除操作

print("Num. features: {}, feature names: {}".format(len(feature_names), feature_names))
#打印feature_names的长度以及该列表的内容
#准备numpy格式的数据，通常用于训练scikit学习模型
X = df[feature_names].to_numpy()#x取df[feature_names]的Numpy数组形式
y = df[label_name].to_numpy().astype(int)#y取将df[feature_names]先转化为Numpy数组类型再转换成整数型
#划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=SEED)

Num. features: 19, feature names: ['duration_in_month', 'credit_history', 'purpose', 'credit_amount', 'savings', 'present_emp_since', 'installment_as_income_perc', 'personal_status_sex', 'other_debtors', 'present_res_since', 'property', 'age', 'other_installment_plans', 'housing', 'credits_this_bank', 'job', 'people_under_maintenance', 'telephone', 'foreign_worker']


### Train the model
Here we train the model, but in a practical situation we may assume that the model has already been trained (and is, e.g., property of the bank that assesses to whether to award the credit or not).

We use random forest because it is quick and easy. However, you can use any model you like, such as a deep neural net. 
As classicly done in XAI litereature, we call this model a *black-box model*. 

In [ ]:
# 训练black-box模型 (bbm)
bbm = RandomForestClassifier(random_state=SEED, class_weight="balanced", min_samples_leaf=25)
#创建随机森林分类器对象，设置随机种子，类别的权重将与它们在数据中出现的频率成反比，较大程度保证模型的效果
#叶子允许有最少25个样本数，否则将不会搜索进一步的拆分
bbm.fit(X_train, y_train)# 在新数据集上训练新分类器
# 注意: 简单起见，我们这里没有对多类别功能进行独热编码

RandomForestClassifier(class_weight='balanced', min_samples_leaf=25,
                       random_state=42)

Let's check that the model has a decent accuracy 
(Note: not really needed for the purpose of CEs)

In [ ]:
print("acc:{:.3f}, bal.-acc:{:.3f}".format(accuracy_score(y_test, bbm.predict(X_test)), balanced_accuracy_score(y_test, bbm.predict(X_test))))
#accuracy_score：基于真实标签y_test和预测标签predict(X_test)的匹配程度计算分类的准确性，
#balanced_accuracy_score：计算平衡精度,避免对不平衡数据集进行夸大的性能估计

acc:0.760, bal.-acc:0.691


### Pick the user
Next, we simulate to have a user for whom the decision of the black-box model is the undesired one. 
For example, let's pick the last point in the test set for which the prediction is unfavourable.

In [ ]:
#当最后一个测试样本不是我们想要的
p = bbm.predict(X_test)#预测
idx = np.argwhere(p != desired_class).squeeze()[-1]
#返回非0的数组元组的索引，索引数组的条件是p不是我们想要的，并移除数组中维度为1的维度
x = X_test[idx] #这是我们不满意的用户

print("Description of x:")
for i, feat_name in enumerate(feature_names):
  print(" ", feat_name+" "*(30-len(feat_name)), x[i])#显示此用户的功能

Description of x:
  duration_in_month              48
  credit_history                 0
  purpose                        8
  credit_amount                  3844
  savings                        2
  present_emp_since              4
  installment_as_income_perc     4
  personal_status_sex            2
  other_debtors                  0
  present_res_since              4
  property                       4
  age                            34
  other_installment_plans        2
  housing                        3
  credits_this_bank              1
  job                            2
  people_under_maintenance       1
  telephone                      0
  foreign_worker                 1


### CE discovery algorithm
We use the library CoGS to find a CE.
CoGS (Counterfactual Genetic Search) is a relatively quick to run and easy to use library that makes no assumptions on the black-box model $f$ (e.g., it does not require linearity nor gradients to work). 
Moreover, CoGS can handle both numerical and categorical features.


### Setting up the search space
To set up the space in which CoGS searches, we must provide:
1) Intervals within which the search takes place (for categorical features, which categories are possible)
2) The indices of categorical features (for CoGS to know which are categorical and which are numerical)
3) Optional plausibility constraints to ensure that the discovered CE can be realized (e.g., the age of a person cannot decrease)

All of these three must be provided as lists that have the same order, in particular the order used to list the feature in `X_train` and `X_test`.

In [ ]:
#设置搜索边界
feature_intervals = list()#设置一个空列表
for i, feat in enumerate(feature_names):#遍历feature_names
  if feat in categorical_feature_names:#如果feat不在categorical_feature_names里面
    interval_i = np.unique(X_train[:,i])#interval_i取去除重复元素的所有行的第i个数据
  else:
    interval_i = (np.min(X_train[:,i]), np.max(X_train[:,i]))#interval_i取所有行第i个数据的最小值和最大值
  feature_intervals.append(interval_i)#将interval_i添加到列表中

#设置哪些特征索引是分类的
indices_categorical_features = [i for i, feat in enumerate(feature_names) if feat in categorical_feature_names]

#让我们还为'age'这一特征设置一个合理性约束（只允许增加）
#另一个是针对foreign_worker的（不允许改变，只能保持现状）
pcs = ['>=' if feat=='age' else ('=' if feat=='foreign_worker' else None) for feat in feature_names]

## Hyper parameters
We can now setup the hyper-parameters of CoGS, and then run the search!
We put some comments to explain what they mean in the code below.

As distance $\delta$, here we use Gower's distance, which takes into account both numerical differences and categorical mismatches (see https://christophm.github.io/interpretable-ml-book/counterfactual.html#method-by-dandl-et-al.).
In a genetic algorithm, the quality of solutions is measured in terms of *fitness*, where normally higher is better.
Thus the fitness used here is set to be the opposite of Gower's distance.

In [ ]:
from cogs.evolution import Evolution#进化算法是一种用于全局优化问题的演化算法
from cogs.fitness import gower_fitness_function

cogs = Evolution(
        #问题的超参数（必需）
        x=x,  #开始的一点，即不满意的用户
        fitness_function=gower_fitness_function, #反事实解释的经典适应度函数
        fitness_function_kwargs={'blackbox':bbm,'desired_class': desired_class},  #这些必须通过，才能使适应度函数发挥作用
        feature_intervals=feature_intervals, #搜索操作的时间间隔
        indices_categorical_features=indices_categorical_features, #分类特征的索引
        plausibility_constraints=pcs, #如果不需要设置约束，则可以为“None”
        #进化的超参数（全部可选）
        evolution_type='classic', #进化的类型为classic
        population_size=1000, #1000个候选的反事实例子可以同时发展
        n_generations=25, #进化的迭代次数为25
        selection_name='tournament_4',  #设置名称
        init_temperature=0.8, #我们初始化离x有0.8
        num_features_mutation_strength=0.25, #数值特征的随机突变强度为0.25
        num_features_mutation_strength_decay=0.5, #超参数的衰减
        num_features_mutation_strength_decay_generations=[10,15,20], 
        #其他设置
        verbose=True #记录每一次迭代的进度
)

Ready to run!

In [ ]:
cogs.run()#运行

generation: 1 best fitness: -0.23960241235590057 avg. fitness: -0.5655859030567522
generation: 2 best fitness: -0.23960241235590057 avg. fitness: -0.45144422711053095
generation: 3 best fitness: -0.23960241235590057 avg. fitness: -0.3932625759375893
generation: 4 best fitness: -0.18999766508484883 avg. fitness: -0.3424200187317699
generation: 5 best fitness: -0.18999766508484883 avg. fitness: -0.29326014109826054
generation: 6 best fitness: -0.10306974074439322 avg. fitness: -0.24382975966224887
generation: 7 best fitness: -0.06996051106514789 avg. fitness: -0.1966318789656929
generation: 8 best fitness: -0.059808815825190084 avg. fitness: -0.15648801037922425
generation: 9 best fitness: -0.03346219281443785 avg. fitness: -0.12237121200700284
generation: 10 best fitness: -0.029642499013702472 avg. fitness: -0.09555415482021212
generation: 11 best fitness: -0.029642499013702472 avg. fitness: -0.0736235690606997
generation: 12 best fitness: -0.029642499013702472 avg. fitness: -0.05854284

## Counterfactual explanation
Now that CoGS has terminated, we can look at its result.
The field `cogs.elite` contains the best-found counterfactual example, i.e., a point `x'` for which `bbm(x')=desired_class`.
The respective counterfactual explanation is simply `x'-x` (there exist more involved definitions of counterfactual explanations, here we use this simple one).
Let's take a look at what the user needs to do to obtain the desired class, i.e., be granted the loan.

In [ ]:
from pandas.core.arrays import categorical #处理离散分类概率分布
#获取最佳反事实示例
cf_example = cogs.elite#字段'cogs.ellite'包含找到最好的反事实例子
cf_explanation = cogs.elite - x#相应的反事实解释
#展示反事实解释
if bbm.predict([cf_example])[0] == desired_class:#如果反事实示例的预测标签是我们想要的
  print("Success! Here's the explanation:")#显示成功
  for i, feat in enumerate(feature_names):#feature_names列表遍历
    if cf_explanation[i] != 0:#将反事实解释不为0的打印出来
      print(" Feature '{}' should change from '{}' to '{}'".format(feat, np.round(x[i],3), np.round(cf_example[i],3)))
else:#否则，显示失败
  print("Failed to find a counterfactual explanation for the desired class :(")

Success! Here's the explanation:
 Feature 'savings' should change from '2' to '2.5'


# Exercise idea
Here's an idea for an exercise.
One of the features is called `foreign_worker`. This may be considered a sensitive feature: should $f$ be allowed to discriminate based only on that?

Try to use CoGS to search whether a CE can be found for (one or more) `x` who is a foreign worker and for whom `bbm` says `high risk`, that recommends not to be a foreign worker.
To do that, you can set the plausibility constraints to "`=`" for all features except for `foreign_worker`.